In [1]:
import tensorflow as tf

# Checks for available GPUs and lists them
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Set TensorFlow to use only the first GPU
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPU, 1 Logical GPU


2024-01-04 12:34:15.963516: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-01-04 12:34:15.963532: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-01-04 12:34:15.963535: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-01-04 12:34:15.963578: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-04 12:34:15.963739: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0


In [6]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

# Reshape data for SVD
x_train_flat = x_train.reshape(x_train.shape[0], -1)
x_test_flat = x_test.reshape(x_test.shape[0], -1)

# Experiment with different numbers of components
n_components = 25  # for example, try 100, 150, 200, etc.

# Apply SVD
svd = TruncatedSVD(n_components=n_components)
x_train_svd = svd.fit_transform(x_train_flat)
x_test_svd = svd.transform(x_test_flat)

# Reshape data back for the neural network input
x_train_svd = x_train_svd.reshape(x_train.shape[0], n_components, 1, 1)
x_test_svd = x_test_svd.reshape(x_test.shape[0], n_components, 1, 1)


In [7]:
# Adjusted model for SVD output
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(n_components, 1, 1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [8]:
model.fit(x_train_svd, y_train, epochs=5)
model.evaluate(x_test_svd, y_test)


Epoch 1/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.5582 - accuracy: 0.8313
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4570 - accuracy: 0.8632
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4649 - accuracy: 0.8627
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4716 - accuracy: 0.8613
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.4018 - accuracy: 0.8829


[0.4017954468727112, 0.8828999996185303]